# RBRP Dry Protocol - バイオインフォマティクス解析自動化パイプライン

このノートブックは、Eric Koolラボの論文「Reactivity-based RNA profiling for analyzing transcriptome interactions of small molecules in human cells」のドライプロトコル部分を自動化します。

**対象ユーザー**: バイオインフォマティクス初学者  
**入力**: FASTQファイル  
**出力**: RNA-薬物相互作用解析結果

## 使用方法
1. 設定セクションで入力ファイルパスを指定
2. 「Run All Cells」で全自動実行
3. 結果は`data/results/`フォルダに保存されます

## 1. 環境設定と初期化

In [ ]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import logging
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートディレクトリを設定
PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT / 'src'))

# ログ設定
log_file = PROJECT_ROOT / 'logs' / f'rbrp_pipeline_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print(f"🧬 RBRP Dry Protocol Pipeline Started")
print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📝 Log File: {log_file}")

## 2. 設定ファイル読み込み・入力ファイル指定

In [ ]:
# ===== ユーザー設定 =====
# ここで入力ファイルを指定してください

# 入力FASTQファイルのパス（複数サンプル対応）
INPUT_FASTQ_FILES = {
    'sample1_probe_only': '/path/to/sample1_probe_only.fastq',
    'sample1_probe_drug': '/path/to/sample1_probe_drug.fastq',
    'sample2_DMSO_ctrl': '/path/to/sample2_DMSO_ctrl.fastq',
    'sample2_drug_ctrl': '/path/to/sample2_drug_ctrl.fastq'
}

# 参照ゲノムファイル
REFERENCE_FILES = {
    'genome_fasta': '/path/to/genome.fa',
    'genome_gtf': '/path/to/genome.gtf',
    'transcriptome_index': '/path/to/transcriptome_index',
    'adapter_fasta': '/path/to/adapter.fa'
}

# バーコード情報
BARCODES = {
    'sample1_probe_only': 'GGTT',
    'sample1_probe_drug': 'TTGT', 
    'sample2_DMSO_ctrl': 'ACCT',
    'sample2_drug_ctrl': 'CAAT'
}

# 解析パラメータ
ANALYSIS_PARAMS = {
    'min_read_length': 25,
    'min_rpkm': 1,
    'min_sequencing_depth': 200,
    'rbrp_score_threshold': 0.12,
    'p_value_threshold': 0.05
}

print("✅ 設定完了")
print(f"📊 解析対象サンプル数: {len(INPUT_FASTQ_FILES)}")
logger.info(f"Analysis started with {len(INPUT_FASTQ_FILES)} samples")

## 3. 依存関係チェック・外部ツール確認

In [ ]:
def check_external_tools():
    """外部ツールの存在確認"""
    required_tools = [
        'fastqc',
        'bowtie2',
        'gffread',
        'wiggletools',
        'bedGraphToBigWig',
        'wigToBigWig'
    ]
    
    missing_tools = []
    
    for tool in required_tools:
        try:
            result = subprocess.run(['which', tool], capture_output=True, text=True)
            if result.returncode == 0:
                print(f"✅ {tool}: {result.stdout.strip()}")
            else:
                missing_tools.append(tool)
                print(f"❌ {tool}: 見つかりません")
        except Exception as e:
            missing_tools.append(tool)
            print(f"❌ {tool}: エラー - {e}")
    
    if missing_tools:
        print(f"\n⚠️ 以下のツールが不足しています: {', '.join(missing_tools)}")
        print("インストール方法はREADME.mdを参照してください")
        return False
    else:
        print("\n🎉 すべての必要ツールが利用可能です")
        return True

tools_available = check_external_tools()

## 4. パイプライン実行関数群

In [ ]:
def run_command(cmd, description, check=True):
    """コマンド実行のヘルパー関数"""
    logger.info(f"実行中: {description}")
    logger.debug(f"コマンド: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, check=check, 
                              capture_output=True, text=True)
        if result.returncode == 0:
            logger.info(f"✅ 完了: {description}")
            return result
        else:
            logger.error(f"❌ 失敗: {description}")
            logger.error(f"エラー出力: {result.stderr}")
            if check:
                raise subprocess.CalledProcessError(result.returncode, cmd)
            return result
    except Exception as e:
        logger.error(f"❌ 例外発生: {description} - {e}")
        if check:
            raise
        return None

def create_output_dirs():
    """出力ディレクトリ作成"""
    dirs = [
        'data/processed/fastqc',
        'data/processed/trimmed', 
        'data/processed/aligned',
        'data/processed/rbrp_scores',
        'data/results/bigwig',
        'data/results/figures'
    ]
    
    for dir_path in dirs:
        (PROJECT_ROOT / dir_path).mkdir(parents=True, exist_ok=True)
    
    logger.info("📁 出力ディレクトリを作成しました")

create_output_dirs()
print("📁 出力ディレクトリ準備完了")

## 5. ステップ1: 品質管理・デマルチプレックス

In [ ]:
def step1_quality_control_and_demultiplex():
    """ステップ1: 品質管理とデマルチプレックス"""
    print("\n🔍 ステップ1: 品質管理・デマルチプレックス開始")
    
    for sample_name, fastq_path in tqdm(INPUT_FASTQ_FILES.items(), desc="Processing samples"):
        if not Path(fastq_path).exists():
            logger.warning(f"⚠️ ファイルが見つかりません: {fastq_path}")
            continue
            
        # FastQC実行
        fastqc_cmd = f"fastqc -o {PROJECT_ROOT}/data/processed/fastqc {fastq_path}"
        run_command(fastqc_cmd, f"FastQC for {sample_name}")
        
        # デマルチプレックス（バーコードが指定されている場合）
        if sample_name in BARCODES:
            barcode = BARCODES[sample_name]
            output_file = PROJECT_ROOT / f"data/processed/{sample_name}_demux.fastq"
            
            # 簡単なバーコード抽出（実際のsplitFastq.plの代替）
            demux_cmd = f"""grep -A 3 "^@.*{barcode}" {fastq_path} | grep -v "^--$" > {output_file}"""
            run_command(demux_cmd, f"Demultiplexing {sample_name}", check=False)
            
            logger.info(f"✅ {sample_name}: デマルチプレックス完了")
    
    print("✅ ステップ1完了: 品質管理・デマルチプレックス")

step1_quality_control_and_demultiplex()

## 6. ステップ2: PCR重複除去・トリミング

In [ ]:
def step2_remove_duplicates_and_trim():
    """ステップ2: PCR重複除去とトリミング"""
    print("\n✂️ ステップ2: PCR重複除去・トリミング開始")
    
    for sample_name in tqdm(INPUT_FASTQ_FILES.keys(), desc="Processing trimming"):
        input_file = PROJECT_ROOT / f"data/processed/{sample_name}_demux.fastq"
        
        if not input_file.exists():
            logger.warning(f"⚠️ 入力ファイルが見つかりません: {input_file}")
            continue
        
        # PCR重複除去（簡易版）
        rmdup_file = PROJECT_ROOT / f"data/processed/{sample_name}_rmdup.fastq"
        rmdup_cmd = f"""awk '/^@/ {{if (seen[$0]++) next}} 1' {input_file} > {rmdup_file}"""
        run_command(rmdup_cmd, f"Remove duplicates for {sample_name}")
        
        # トリミング
        trimmed_file = PROJECT_ROOT / f"data/processed/trimmed/{sample_name}_trimmed.fastq"
        
        # アダプター配列除去とクオリティトリミング
        trim_cmd = f"""cutadapt -a AGATCGGAAGAGCGGTTCAG -q 30 -m {ANALYSIS_PARAMS['min_read_length']} \
                     -o {trimmed_file} {rmdup_file}"""
        
        run_command(trim_cmd, f"Trimming adapters for {sample_name}", check=False)
        logger.info(f"✅ {sample_name}: トリミング完了")
    
    print("✅ ステップ2完了: PCR重複除去・トリミング")

step2_remove_duplicates_and_trim()

## 7. ステップ3: 配列アライメント・転写産物発現量計算

In [ ]:
def step3_alignment_and_expression():
    """ステップ3: 配列アライメントと転写産物発現量計算"""
    print("\n🧬 ステップ3: アライメント・発現量計算開始")
    
    for sample_name in tqdm(INPUT_FASTQ_FILES.keys(), desc="Processing alignment"):
        trimmed_file = PROJECT_ROOT / f"data/processed/trimmed/{sample_name}_trimmed.fastq"
        
        if not trimmed_file.exists():
            logger.warning(f"⚠️ トリミング済みファイルが見つかりません: {trimmed_file}")
            continue
        
        # Bowtie2でアライメント
        sam_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.sam"
        
        align_cmd = f"""bowtie2 -U {trimmed_file} -S {sam_file} \
                       -x {REFERENCE_FILES['transcriptome_index']} \
                       --non-deterministic --time"""
        
        run_command(align_cmd, f"Alignment for {sample_name}")
        
        # RPKM計算
        rpkm_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rpkm"
        rpkm_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rpkm.py -i {sam_file} -o {rpkm_file}"""
        
        run_command(rpkm_cmd, f"RPKM calculation for {sample_name}", check=False)
        
        # RTstop計算
        rt_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rt"
        rt_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rtstops.py \
                    -i {sam_file} -o {rt_file} -r {rpkm_file} -c {ANALYSIS_PARAMS['min_rpkm']}"""
        
        run_command(rt_cmd, f"RTstop calculation for {sample_name}", check=False)
        logger.info(f"✅ {sample_name}: アライメント・発現量計算完了")
    
    print("✅ ステップ3完了: アライメント・発現量計算")

step3_alignment_and_expression()

## 8. ステップ4: RBRPスコア計算・統計解析

In [ ]:
def step4_rbrp_score_calculation():
    """ステップ4: RBRPスコア計算と統計解析"""
    print("\n📊 ステップ4: RBRPスコア計算・統計解析開始")
    
    # サンプルをグループ分け
    probe_samples = [k for k in INPUT_FASTQ_FILES.keys() if 'probe' in k]
    control_samples = [k for k in INPUT_FASTQ_FILES.keys() if 'ctrl' in k or 'DMSO' in k]
    
    print(f"📋 プローブサンプル: {probe_samples}")
    print(f"📋 コントロールサンプル: {control_samples}")
    
    # バックグラウンドRTstopファイルをマージ
    if len(control_samples) >= 2:
        control_files = [f"{PROJECT_ROOT}/data/processed/aligned/{s}.rt" for s in control_samples]
        merged_control = PROJECT_ROOT / "data/processed/rbrp_scores/merged_control.rt"
        
        merge_cmd = f"""python {PROJECT_ROOT}/scripts/merge_rt_files.py \
                       -i {':'.join(control_files)} -o {merged_control}"""
        run_command(merge_cmd, "Merging control RTstop files", check=False)
    
    # 各プローブサンプルのRBRPスコア計算
    for sample_name in tqdm(probe_samples, desc="Calculating RBRP scores"):
        rt_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rt"
        
        if not rt_file.exists():
            logger.warning(f"⚠️ RTファイルが見つかりません: {rt_file}")
            continue
        
        # RTファイル正規化
        normalized_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_normalized.rt"
        norm_cmd = f"""python {PROJECT_ROOT}/scripts/normalize_rt_file.py \
                      -i {rt_file} -o {normalized_file} -d 32 -l 32"""
        run_command(norm_cmd, f"Normalizing RT file for {sample_name}", check=False)
        
        # RBRPスコア計算
        rbrp_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_rbrp.out"
        background_file = merged_control if 'merged_control' in locals() else None
        
        if background_file and background_file.exists():
            rbrp_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rbrp_score.py \
                          -f {normalized_file} -b {background_file} -o {rbrp_file} \
                          -e dividing -y 0.5"""
        else:
            rbrp_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rbrp_score.py \
                          -f {normalized_file} -o {rbrp_file}"""
        
        run_command(rbrp_cmd, f"Calculating RBRP scores for {sample_name}", check=False)
        
        # 低品質スコアフィルタリング
        filtered_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        filter_cmd = f"""python {PROJECT_ROOT}/scripts/filter_rbrp_scores.py \
                        -i {rbrp_file} -o {filtered_file} \
                        -t {ANALYSIS_PARAMS['min_sequencing_depth']} -s 5 -e 30"""
        run_command(filter_cmd, f"Filtering RBRP scores for {sample_name}", check=False)
        
        logger.info(f"✅ {sample_name}: RBRPスコア計算完了")
    
    print("✅ ステップ4完了: RBRPスコア計算・統計解析")

step4_rbrp_score_calculation()

## 9. ステップ5: 可視化ファイル生成・結果出力

In [ ]:
def step5_visualization_and_output():
    """ステップ5: 可視化ファイル生成と結果出力"""
    print("\n📈 ステップ5: 可視化ファイル生成・結果出力開始")
    
    for sample_name in tqdm(probe_samples, desc="Generating visualization files"):
        filtered_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        
        if not filtered_file.exists():
            logger.warning(f"⚠️ フィルタリング済みファイルが見つかりません: {filtered_file}")
            continue
        
        # bedgraphファイル生成
        bedgraph_file = PROJECT_ROOT / f"data/results/bigwig/{sample_name}.bedgraph"
        bedgraph_cmd = f"""python {PROJECT_ROOT}/scripts/generate_bedgraph.py \
                          -i {filtered_file} -o {bedgraph_file} \
                          -g {REFERENCE_FILES['genome_gtf']} \
                          -a {REFERENCE_FILES.get('transcriptome_fasta', '')}"""
        run_command(bedgraph_cmd, f"Generating bedgraph for {sample_name}", check=False)
        
        # bigwigファイル生成（UCscツールが利用可能な場合）
        bigwig_file = PROJECT_ROOT / f"data/results/bigwig/{sample_name}.bw"
        genome_size_file = PROJECT_ROOT / "data/genome.size"  # 事前に準備が必要
        
        if bedgraph_file.exists():
            # ソートと重複除去
            sorted_bedgraph = PROJECT_ROOT / f"data/results/bigwig/{sample_name}_sorted.bedgraph"
            sort_cmd = f"sort -k1,1 -k2,3n {bedgraph_file} | uniq > {sorted_bedgraph}"
            run_command(sort_cmd, f"Sorting bedgraph for {sample_name}", check=False)
            
            # bigwig変換
            if genome_size_file.exists():
                bw_cmd = f"bedGraphToBigWig {sorted_bedgraph} {genome_size_file} {bigwig_file}"
                run_command(bw_cmd, f"Converting to bigwig for {sample_name}", check=False)
        
        logger.info(f"✅ {sample_name}: 可視化ファイル生成完了")
    
    print("✅ ステップ5完了: 可視化ファイル生成・結果出力")

step5_visualization_and_output()

## 10. 結果サマリー・統計情報

In [ ]:
def generate_summary_report():
    """結果サマリーレポート生成"""
    print("\n📋 結果サマリーレポート生成")
    
    summary_data = []
    
    for sample_name in INPUT_FASTQ_FILES.keys():
        sample_summary = {'sample_name': sample_name}
        
        # 各ステップのファイル存在確認
        files_to_check = {
            'demultiplexed': f"data/processed/{sample_name}_demux.fastq",
            'trimmed': f"data/processed/trimmed/{sample_name}_trimmed.fastq",
            'aligned': f"data/processed/aligned/{sample_name}.sam",
            'rbrp_scores': f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        }
        
        for step, file_path in files_to_check.items():
            full_path = PROJECT_ROOT / file_path
            sample_summary[f'{step}_exists'] = full_path.exists()
            if full_path.exists():
                sample_summary[f'{step}_size_mb'] = round(full_path.stat().st_size / 1024 / 1024, 2)
        
        summary_data.append(sample_summary)
    
    # サマリーDataFrame作成
    summary_df = pd.DataFrame(summary_data)
    
    # 結果表示
    print("\n📊 処理結果サマリー:")
    print(summary_df.to_string(index=False))
    
    # CSVファイルとして保存
    summary_file = PROJECT_ROOT / "data/results/processing_summary.csv"
    summary_df.to_csv(summary_file, index=False)
    
    # 統計情報
    successful_samples = summary_df['rbrp_scores_exists'].sum()
    total_samples = len(summary_df)
    
    print(f"\n📈 処理統計:")
    print(f"   総サンプル数: {total_samples}")
    print(f"   成功サンプル数: {successful_samples}")
    print(f"   成功率: {successful_samples/total_samples*100:.1f}%")
    print(f"   サマリーファイル: {summary_file}")
    
    return summary_df

summary_report = generate_summary_report()

# 実行時間の記録
print(f"\n🎉 パイプライン実行完了!")
print(f"📁 結果ファイルは以下に保存されました:")
print(f"   - 処理済みデータ: {PROJECT_ROOT}/data/processed/")
print(f"   - 最終結果: {PROJECT_ROOT}/data/results/")
print(f"   - ログファイル: {log_file}")

logger.info("RBRP Dry Protocol Pipeline completed successfully")

## 11. 結果可視化（オプション）

In [ ]:
def create_visualization_plots():
    """結果の可視化プロット作成"""
    print("\n📊 結果可視化プロット作成")
    
    # 処理サマリーの可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('RBRP Dry Protocol - 処理結果サマリー', fontsize=16, y=0.98)
    
    # 1. ファイルサイズ分布
    size_columns = [col for col in summary_report.columns if 'size_mb' in col]
    if size_columns:
        size_data = summary_report[size_columns].fillna(0)
        axes[0, 0].bar(range(len(size_columns)), size_data.mean(), 
                      tick_label=[col.replace('_size_mb', '') for col in size_columns])
        axes[0, 0].set_title('平均ファイルサイズ (MB)')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. 処理成功率
    success_columns = [col for col in summary_report.columns if 'exists' in col]
    if success_columns:
        success_rates = summary_report[success_columns].mean() * 100
        axes[0, 1].bar(range(len(success_rates)), success_rates.values,
                      tick_label=[col.replace('_exists', '') for col in success_columns])
        axes[0, 1].set_title('処理成功率 (%)')
        axes[0, 1].set_ylim(0, 100)
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. サンプル別処理状況
    if success_columns:
        sample_success = summary_report[success_columns].sum(axis=1)
        axes[1, 0].bar(range(len(sample_success)), sample_success.values,
                      tick_label=summary_report['sample_name'])
        axes[1, 0].set_title('サンプル別完了ステップ数')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. 処理時間の目安（仮想データ）
    processing_steps = ['デマルチプレックス', 'トリミング', 'アライメント', 'RBRP計算', '可視化']
    estimated_times = [5, 10, 30, 20, 5]  # 分単位
    axes[1, 1].bar(processing_steps, estimated_times)
    axes[1, 1].set_title('ステップ別推定処理時間 (分)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    # 図を保存
    plot_file = PROJECT_ROOT / "data/results/figures/processing_summary.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 可視化プロット保存: {plot_file}")

# 可視化実行
try:
    create_visualization_plots()
except Exception as e:
    logger.warning(f"可視化プロット作成エラー: {e}")
    print("⚠️ 可視化プロットの作成でエラーが発生しましたが、パイプライン自体は正常に完了しています")